In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)                                      # see all the rows and columns completely

# ---------------------------------------------
airports         = pd.read_csv("/content/airports.csv")   
flights_2018     = pd.read_csv("/content/2008.csv")                             # data about the flights in 2018
plane_data       = pd.read_csv("/content/plane-data.csv")                       # could be useful to understand the size of the plane (estimates of the number of passengers)
var_descriptions = pd.read_csv("/content/variable-descriptions.csv")            # description of the variables of flights_2018
carriers         = pd.read_csv("/content/carriers.csv")

# ---------------------------------------------
cities_to_county = pd.read_csv("/content/uscities.csv")                         # https://simplemaps.com/data/us-cities
# iata_to_fips     = pd.read_csv("/content/T_T100D_MARKET_ALL_CARRIER.csv")
# ---------------------------------------------
counties_2020 = pd.read_csv("/content/us-counties-2020.csv")
counties_2021 = pd.read_csv("/content/us-counties-2021.csv")
counties_2022 = pd.read_csv("/content/us-counties-2022.csv")

# important note: use fips and not the name of the county. there could be duplicates in names

In [5]:
flights_2018.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,1343.0,1325,1451.0,1435,WN,588,...,4.0,9.0,0.0,NaN,0.0,16.0,0.0,0.0,0.0,0.0
1,2008,1,3,4,1125.0,1120,1247.0,1245,WN,1343,...,3.0,8.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,2009.0,2015,2136.0,2140,WN,3841,...,2.0,14.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,903.0,855,1203.0,1205,WN,3,...,5.0,7.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1423.0,1400,1726.0,1710,WN,25,...,6.0,10.0,0.0,NaN,0.0,16.0,0.0,0.0,0.0,0.0


In [6]:
# interesting variables: Origin, Dest --> origin IATA airport code, destination IATA airport code
flights_2018_v1 = flights_2018[["Origin", "Dest"]]
flights_2018_v1.head()

,Origin,Dest
0,HOU,LIT
1,HOU,MAF
2,HOU,MAF
3,HOU,MCO
4,HOU,MCO


In [7]:
# the following dataset show for each air route the number on flights
flights_2018_v2 = flights_2018_v1.groupby(["Origin", "Dest"]).size().reset_index(name="Count")
flights_2018_v2.head()

,Origin,Dest,Count
0,ABE,ATL,156
1,ABE,CLE,146
2,ABE,CLT,60
3,ABE,CVG,39
4,ABE,DTW,85


In [8]:
airports.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [9]:
# PROBLEM: O(N^2) COMPLEXITY... IT TAKES FOREVER --> SOLUTION: CREATE A DICTIONARY BASED ON UNIQUE VALUES

# airports X flights
# adding airport name, city, state, country, lat and long of both departure and destination to the flights_2018 dataset --> based on each iata code present in airports

flights_2018_v2["Origin_airport"] = ""
flights_2018_v2["Origin_city"] = ""
flights_2018_v2["Origin_state"] = ""
flights_2018_v2["Origin_lat"] = ""
flights_2018_v2["Origin_long"] = ""

flights_2018_v2["Dest_airport"] = ""
flights_2018_v2["Dest_city"] = ""
flights_2018_v2["Dest_state"] = ""
flights_2018_v2["Dest_lat"] = ""
flights_2018_v2["Dest_long"] = ""

flights_2018_v2.head()

for index, row in flights_2018_v2.iterrows():
  found = False
  for index1, row1 in airports.iterrows():
    if row["Origin"] == row1["iata"]:
      flights_2018_v2.Origin_airport.iloc[index] = row1["airport"]
      flights_2018_v2.Origin_city.iloc[index] = row1["city"]
      flights_2018_v2.Origin_state.iloc[index] = row1["state"]
      flights_2018_v2.Origin_lat.iloc[index] = row1["lat"]
      flights_2018_v2.Origin_long.iloc[index] = row1["long"]
      # print(flights_2018_v2.head()) # just to check the progess
      # print(index)                  # just to check the progess
      found = True
      break
  # in case the iata code is not available in the iata_city dataset
  if found == False: 
    print(row["Origin"])
    break    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
flights_2018_v2.head()

,Origin,Dest,Count,Origin_airport,Origin_city,Origin_state,Origin_lat,Origin_long,Dest_airport,Dest_city,Dest_state,Dest_lat,Dest_long
0,ABE,ATL,156,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,,,,,
1,ABE,CLE,146,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,,,,,
2,ABE,CLT,60,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,,,,,
3,ABE,CVG,39,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,,,,,
4,ABE,DTW,85,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,,,,,


In [11]:
# PROBLEM: O(N^2) COMPLEXITY... IT TAKES FOREVER

# same procedure for the Dest column
for index, row in flights_2018_v2.iterrows():
  found = False
  for index1, row1 in airports.iterrows():
    if row["Dest"] == row1["iata"]:
      flights_2018_v2.Dest_airport.iloc[index] = row1["airport"]
      flights_2018_v2.Dest_city.iloc[index] = row1["city"]
      flights_2018_v2.Dest_state.iloc[index] = row1["state"]
      flights_2018_v2.Dest_lat.iloc[index] = row1["lat"]
      flights_2018_v2.Dest_long.iloc[index] = row1["long"]
      found = True
      break
  # in case the iata code is not available in the iata_city dataset
  if found == False: 
    print(row["Origin"])
    break    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
flights_2018_v2.head()

,Origin,Dest,Count,Origin_airport,Origin_city,Origin_state,Origin_lat,Origin_long,Dest_airport,Dest_city,Dest_state,Dest_lat,Dest_long
0,ABE,ATL,156,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944
1,ABE,CLE,146,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Cleveland-Hopkins Intl,Cleveland,OH,41.410894,-81.849397
2,ABE,CLT,60,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Charlotte/Douglas International,Charlotte,NC,35.214011,-80.943126
3,ABE,CVG,39,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Cincinnati Northern Kentucky Intl,Covington,KY,39.046143,-84.662172
4,ABE,DTW,85,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Detroit Metropolitan-Wayne County,Detroit,MI,42.212059,-83.348836


In [13]:
# PROBLEM: based on the city add the county of departure and the one of arrival --> more county could have cities sharing the name
# CHANGE: O(N^2) COMPLEXITY... IT TAKES FOREVER

# # note: in a state no city has the same name

# cities_to_county.head()
# cities_to_county_v1 = cities_to_county[["city_ascii", "state_id", "county_fips", "county_name"]]
# cities_to_county_v1.head()


# flights_2018_v2["Origin_county_fips"] = ""
# flights_2018_v2["Origin_county_name"] = ""

# flights_2018_v2["Dest_county_fips"] = ""
# flights_2018_v2["Dest_county_name"] = ""

# flights_2018_v2.head()

# for index, row in flights_2018_v2.iterrows():
#   found = False
#   for index1, row1 in cities_to_county_v1.iterrows():
#     if row["Origin_state"] == row1["state_id"] and row["Origin_city"] == row1["city_ascii"]:
#       found = True
#       flights_2018_v2.Origin_county_fips.iloc[index] = row1["county_fips"]
#       flights_2018_v2.Origin_county_name.iloc[index] = row1["county_name"]
#       break
#   if found == False:
#     print("a) \n",
#           row)

# flights_2018_v2.head()

# flights_2018_v3 = flights_2018_v2.groupby(["Origin_county_fips", "Dest_county_fips"]).size().reset_index(name="Count")
# flights_2018_v3.head()

In [17]:
# finding county fips based on the following API
# attempt using https://geo.fcc.gov/api/census/#!/block/get_block_find API

# idea for implementaion
# https://colab.research.google.com/github/nestauk/im-tutorials/blob/3-ysi-tutorial/notebooks/APIs/API_tutorial.ipynb#scrollTo=O_4T8cKVNjmh

# request to the API
# https://geo.fcc.gov/api/census/block/find?latitude=31.953765&longitude=-89.234505&censusYear=2020&format=xml

# other utilities
# df = pd.read_json(filename)

# read json from link url

# import requests  # Import the requests library

# # Query URL
# url = ('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en'
#        '/nama_10_gdp?precision=1'
#        '&unit=CLV05_MEUR'  # Unit: CLV (2005) Million EUR
#        '&geo=NL&geo=DE'  # Country: Netherlands, Germany
#        '&time=2010&time=2011&time=2012'  # Years: 2010, 2011, 2012
#        '&na_item=B1GQ&na_item=D21'  # GDP (market prices) & taxes on products
#        )
# # Some api's will have nicer syntax like:
# # `&time=2010..2012` or `&na_item=B1GQ,D21`
# print(url)

# response = requests.get(url)  # Make a GET request to the URL

# # Print status code (and associated text)
# print(f"Request returned {response.status_code} : '{response.reason}'")

# # Print data returned (parsing as JSON)
# payload = response.json()  # Parse `response.text` into JSON


# import pprint
# pp = pprint.PrettyPrinter(indent=1)
# pp.pprint(payload)
# # NOTE: Could use print(response.json()) but this wouldn't be formatted nicely

# ------------------------------------------------------------------------------

import requests  # Import the requests library
import io
import json

def latlong_to_fips(lat,long):

  url = ("https://geo.fcc.gov/api/census/block/find?latitude="
         f"{lat}"
         "&longitude="
         f"{long}"
         "&censusYear=2010&format=xml")
  response = requests.get(url)
  # print(f"Request returned {response.status_code} : '{response.reason}'")
  # print(response.text)
  # print(type(response.text))

  text = response.text

  t = text.split("><")
  # t

  for el in t:
    if el[:11] == "County FIPS":
      el = el.split()
      el = el[1].split('"')
      el = el[1]
      return el

In [19]:
unique_iata = {}

# dict: iata coord

for index, row in flights_2018_v2.iterrows():
  if flights_2018_v2.Origin.iloc[index] not in unique_iata:
    fips = latlong_to_fips(flights_2018_v2.Origin_lat.iloc[index],flights_2018_v2.Origin_long.iloc[index])
    unique_iata[flights_2018_v2.Origin.iloc[index]] = [flights_2018_v2.Origin_lat.iloc[index], flights_2018_v2.Origin_long.iloc[index], fips]

unique_iata

{'ABE': [40.65236278, -75.44040167, '42077'],
 'ABI': [32.41132, -99.68189722, '48441'],
 'ABQ': [35.04022222, -106.6091944, '35001'],
 'ABY': [31.535515, -84.19447333, '13095'],
 'ACT': [31.61128833, -97.23051917, '48309'],
 'ACV': [40.97811528, -124.1086189, '06023'],
 'ACY': [39.45758333, -74.57716667, '34001'],
 'ADK': [51.87796389, -176.6460306, '02016'],
 'ADQ': [57.74996778, -152.4938553, '02150'],
 'AEX': [31.32737167, -92.54855611, '22079'],
 'AGS': [33.369955, -81.96449611, '13245'],
 'ALB': [42.74811944, -73.80297861, '36001'],
 'ALO': [42.55708139, -92.40034361, '19013'],
 'AMA': [35.2193725, -101.7059272, '48375'],
 'ANC': [61.17432028, -149.9961856, '02020'],
 'ASE': [39.22316, -106.868845, '08097'],
 'ATL': [33.64044444, -84.42694444, '13063'],
 'ATW': [44.25740806, -88.51947556, '55087'],
 'AUS': [30.19453278, -97.66987194, '48453'],
 'AVL': [35.43619444, -82.54180556, '37021'],
 'AVP': [41.33814944, -75.7242675, '42079'],
 'AZO': [42.234875, -85.5520575, '26077'],
 'BD

In [20]:
for index, row in flights_2018_v2.iterrows():
  if flights_2018_v2.Dest.iloc[index] not in unique_iata:
    fips = latlong_to_fips(flights_2018_v2.Dest_lat.iloc[index],flights_2018_v2.Dest_long.iloc[index])
    unique_iata[flights_2018_v2.Dest.iloc[index]] = [flights_2018_v2.Dest_lat.iloc[index], flights_2018_v2.Dest_long.iloc[index], fips]

In [21]:
flights_2018_v2["Origin_county_fips"] = ""
flights_2018_v2["Dest_county_fips"] = ""

for index, row in flights_2018_v2.iterrows():
    flights_2018_v2.Origin_county_fips.iloc[index] = unique_iata[flights_2018_v2.Origin.iloc[index]][2]
    flights_2018_v2.Dest_county_fips.iloc[index] = unique_iata[flights_2018_v2.Dest.iloc[index]][2]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
flights_2018_v2.head()

,Origin,Dest,Count,Origin_airport,Origin_city,Origin_state,Origin_lat,Origin_long,Dest_airport,Dest_city,Dest_state,Dest_lat,Dest_long,Origin_county_fips,Dest_county_fips
0,ABE,ATL,156,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944,42077,13063
1,ABE,CLE,146,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Cleveland-Hopkins Intl,Cleveland,OH,41.410894,-81.849397,42077,39035
2,ABE,CLT,60,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Charlotte/Douglas International,Charlotte,NC,35.214011,-80.943126,42077,37119
3,ABE,CVG,39,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Cincinnati Northern Kentucky Intl,Covington,KY,39.046143,-84.662172,42077,21015
4,ABE,DTW,85,Lehigh Valley International,Allentown,PA,40.652363,-75.440402,Detroit Metropolitan-Wayne County,Detroit,MI,42.212059,-83.348836,42077,26163


In [28]:
flights_2018_v2["fips_to_fips"] = ""

for index, row in flights_2018_v2.iterrows():
    flights_2018_v2.fips_to_fips.iloc[index] = f"{flights_2018_v2.Origin_county_fips.iloc[index]}->{flights_2018_v2.Dest_county_fips.iloc[index]}"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [85]:
flights_2018_v3 = flights_2018_v2.groupby("fips_to_fips").sum("Count")
flights_2018_v3

,Count
fips_to_fips,
01045->13063,228
01073->04013,60
01073->08031,115
01073->12031,60
01073->12057,180
...,...
78030->34013,31
78030->36081,45
78030->37119,75


In [86]:
# e.g. add to the ATL set the ABE county iff the number of flights from ABE to ATL is higher than a given threshold
flights_2018_v3.describe()

,Count
count,4463.000000
mean,211.237957
std,273.452451
min,1.000000
25%,60.000000
50%,120.000000
75%,263.000000
max,2991.000000


In [87]:
# ordering
flights_2018_v3 = flights_2018_v3.sort_values(by=['Count'])
flights_2018_v3.head()

,Count
fips_to_fips,
08031->05119,1
23019->50007,1
56025->16077,1
55087->55079,1
55087->45019,1


In [88]:
# adjancents counties
# 12 per set 
# 3200 counties

percentile_75 = int(flights_2018_v3.Count.quantile(0.75)) # 75th percentile: 263 flights

count = 0

for index, row in flights_2018_v3.iterrows():
  if int(row.Count) > percentile_75:
    count +=1

In [89]:
# number of "adjacent" counties
count

1111

In [90]:
flights_2018_v3 = flights_2018_v3.reset_index()

In [92]:
flights_2018_v3 = flights_2018_v3.loc[flights_2018_v3['Count'] > percentile_75]
flights_2018_v3["origin_fips"] = ""
flights_2018_v3["dest_fips"]   = ""

for index, row in flights_2018_v3.iterrows():
  flights_2018_v3.origin_fips.loc[index] = flights_2018_v3.fips_to_fips.loc[index].split("->")[0]
  flights_2018_v3.dest_fips.loc[index]   = flights_2018_v3.fips_to_fips.loc[index].split("->")[1]

flights_2018_v3.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,fips_to_fips,Count,origin_fips,dest_fips
3352,42045->48201,264,42045,48201
3353,18097->12095,264,18097,12095
3354,26163->48201,265,26163,48201
3355,48201->26163,265,48201,26163
3356,36081->29189,265,36081,29189


In [97]:
# flights_2018_v3 = flights_2018_v3.drop("fips_to_fips", axis = 1)
flights_2018_v3.columns = ['number_flights', 'origin_fips', 'dest_fips']

flights_2018_v3.head()

,number_flights,origin_fips,dest_fips
3352,264,42045,48201
3353,264,18097,12095
3354,265,26163,48201
3355,265,48201,26163
3356,265,36081,29189


In [98]:
flights_2018_v3.to_csv(r'/content/air_mobility.csv', index = False)
